Dit script analyseert de verdeling van elke kolom in een DataFrame om de best passende distributie te vinden. Voor elke kolom worden NaN-waarden en waarden gelijk aan '0' uitgesloten van de analyse. Vervolgens worden verschillende veelvoorkomende statistische distributies, zoals normaal, lognormaal, exponentieel en gamma, aangepast aan de gegevens van elke kolom. De empirische verdeling van de gegevens wordt geplot samen met de geschatte verdelingscurves. Dit stelt de gebruiker in staat om visueel te beoordelen welke distributie het best bij de gegevens van elke kolom past. De code maakt gebruik van 'matplotlib' voor het plotten en 'scipy.stats' voor de distributiefitting.

In [1]:
def analyze_distributions(df, columns, distributions):
    for column in columns:
        column_data = df[column].dropna()
        column_data = column_data[column_data != 0]

        plt.figure(figsize=(10, 5))
        plt.hist(column_data, bins=30, density=True, alpha=0.6, color='g')

        # Initialiseer variabelen om de beste fit te vinden
        best_distribution = None
        best_params = None
        best_sse = np.inf

        for distribution in distributions:
            params = distribution.fit(column_data)

            x = np.linspace(min(column_data), max(column_data), 100)
            plt.plot(x, distribution.pdf(x, *params), label=f'{distribution.name}')

            # Bereken de sum of squared errors
            estimated_pdf = distribution.pdf(column_data, *params)
            sse = np.sum(np.power(column_data - estimated_pdf, 2))

            if sse < best_sse:
                best_sse = sse
                best_distribution = distribution.name
                best_params = params

        plt.title(f'Data versus Distributies voor {column}')
        plt.legend()
        plt.show()

        # Print de samenvatting van de beste fit
        print(f"Beste fit voor {column}: {best_distribution}")
        print(f"  Parameters: {best_params}")
        print(f"  SSE: {best_sse}\n")

Dit script is ontworpen om de lineaire relatie tussen twee specifieke kolommen in een DataFrame te analyseren en te visualiseren. Het focust op de relatie tussen een doelkolom ('target_column') en een vergelijkingskolom ('comparison_column'). De functie plot_linear_relationships verwijdert eerst alle NaN-waarden uit beide kolommen, zorgt ervoor dat ze dezelfde lengte hebben, en maakt vervolgens een scatterplot met een lineaire regressielijn. Daarnaast wordt een samenvatting gegeven die de eigenschappen van de lineaire relatie beschrijft, waaronder de helling, het intercept, de correlatiecoëfficiënt (R-waarde) en de p-waarde. De functie geeft ook een waardeoordeel gebaseerd op de sterkte en significantie van de correlatie, waardoor inzicht wordt verkregen in de aard van de relatie tussen de twee kolommen. Door middel van de 'hue' kan een derde variabelen worden toegevoegd aan de plot, waardoor de gebruiker de relatie tussen de twee kolommen kan visualiseren voor verschillende waarden van de derde variabele.

In [2]:
def plot_linear_relationships(df, target_column, comparison_column, hue_column=None):
    plt.figure(figsize=(15, 12))

    # Bereid de data voor door alleen rijen te behouden waar beide kolommen geldige waarden hebben
    valid_data = df[[target_column, comparison_column]].dropna()

    if hue_column and hue_column in df.columns:
        valid_data = valid_data.join(df[hue_column])

    x = valid_data[target_column]
    y = valid_data[comparison_column]

    # Bereken de lineaire regressielijn
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    # Maak de scatterplot met hue als het is meegegeven
    if hue_column and hue_column in df.columns:
        sns.scatterplot(x=x, y=y, hue=valid_data[hue_column] ,palette='viridis')
    else:
        plt.scatter(x, y, alpha=0.5, label=comparison_column)

    # Voeg de regressielijn toe
    plt.plot(x, intercept + slope * x, 'r', label=f'y={slope:.2f}x+{intercept:.2f}')

    # Voeg titels en legenda toe
    plt.title(f'Lineaire Relatie tussen {target_column} en {comparison_column}')
    plt.xlabel(target_column)
    plt.ylabel(comparison_column)

    plt.show()

    # Print de samenvatting van de relatie
    print(f"Samenvatting voor {comparison_column}:")
    print(f"  Helling (Slope): {slope:.2f}")
    print(f"  Intercept: {intercept:.2f}")
    print(f"  Correlatiecoëfficiënt (R-waarde): {r_value:.2f}")
    print(f"  P-waarde: {p_value:.2f}")
    
    # Waardeoordeel gebaseerd op R-waarde en P-waarde
    if p_value < 0.05:
        if abs(r_value) > 0.7:
            print("  Sterke en significante relatie.")
        elif abs(r_value) > 0.3:
            print("  Matige, maar significante relatie.")
        else:
            print("  Zwakke, maar significante relatie.")
    else:
        print("  Geen significante relatie.")
    print("\n")


Deze functie, plot_corr, is ontwikkeld om een correlatieheatmap te genereren voor een gegeven pandas DataFrame. De functie biedt de flexibiliteit om een specifieke subset van kolommen (cols) te selecteren voor de correlatieanalyse. Als geen kolommen worden opgegeven, analyseert de functie alle kolommen in het DataFrame. De heatmap visualiseert de Pearson-correlatiecoëfficiënten tussen de kolommen, waarbij sterkere correlaties worden weergegeven in meer intense kleuren. De functie maakt gebruik van Seaborn's heatmap met een divergerende kleurenpalet voor duidelijke visualisatie van positieve en negatieve correlaties. Parameters zoals vmax (vm) bieden controle over de kleurenschaal van de heatmap, waarmee de gebruiker kan bepalen hoe correlatiewaarden worden gemapt naar de kleurenschaal. Deze functie is bijzonder nuttig voor het verkennen van relaties tussen variabelen in een dataset, wat essentieel is voor data-analyse en feature selectie processen.


In [3]:
def plot_corr(df, cols=None, vm=1.0):
    # Creëert een nieuwe figuur en assen voor de plot
    fig, ax = plt.subplots()

    # Als geen specifieke kolommen zijn opgegeven, gebruik dan alle kolommen in het DataFrame
    if cols is None:
        cols = df.columns

    # Bereken de correlatiematrix van de geselecteerde kolommen
    corr = df[cols].corr()

    # Maak een masker voor het bovenste driehoekige deel van de heatmap, om duplicatie te voorkomen
    mask = np.zeros_like(corr, dtype=bool)
    mask[np.triu_indices_from(mask)] = True

    # Definieer een kleurenpalet voor de heatmap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Creëer de heatmap van de correlatiematrix
    sns.heatmap(corr, 
                #mask=mask,  # Optie om het masker toe te passen om alleen het onderste driehoekige deel te tonen
                cmap=cmap,  # Het gekozen kleurenpalet
                fmt=".2f",  # Format voor de getallen in de cellen
                vmax=vm,    # Maximumwaarde voor de kleurenschaal
                vmin=-vm,   # Minimumwaarde voor de kleurenschaal
                center=0,   # Centrumwaarde voor de kleurenschaal, waarbij wit meestal de kleur van het midden is
                square=True,  # Zorgt ervoor dat elke cel vierkant is
                linewidths=.5,  # Breedte van de lijnen die de cellen scheiden
                cbar_kws={"shrink": .5},  # Extra opties voor de legenda, hier wordt het met 50% verkleind
                ax=ax  # De assen waarop de heatmap geplot wordt
               )

    # Retourneer de correlatiematrix en de assen
    return corr, ax


Deze functie, analyze_data_distribution, is ontworpen om de verdeling van data in specifieke kolommen van een pandas DataFrame te analyseren. Voor elke opgegeven kolom worden eerst NaN-waarden en waarden gelijk aan '0' verwijderd. Vervolgens genereert de functie voor elke kolom een histogram en een Quantile-Quantile (QQ) plot om visueel te beoordelen of de gegevens een normale verdeling volgen. Daarnaast voert de functie een Shapiro-Wilk test uit om de normaliteit van de gegevens statistisch te toetsen. Deze test is beperkt tot 5000 waarnemingen per kolom; indien een kolom meer waarnemingen bevat, wordt een steekproef genomen. De functie is nuttig voor het preliminair onderzoeken van de gegevensdistributies in een dataset, wat cruciaal kan zijn voor verdere data-analyse en modelselectie.

In [4]:
def analyze_data_distribution(df, headers, min_value=-0.5, max_value=0.5):
    for column in headers:
        # Verwijder NaN waarden en waarden die gelijk zijn aan '0' uit de kolom voor de analyse
        column_data = df[column].dropna()
        column_data = column_data[(column_data < min_value) | (column_data > max_value)]

        # Histogram
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        column_data.hist(bins=30)
        plt.title(f'Histogram van {column}')

        # QQ-plot
        plt.subplot(1, 2, 2)
        stats.probplot(column_data, dist="norm", plot=plt)
        plt.title(f'QQ-plot van {column}')

        plt.tight_layout()
        plt.show()

        # Shapiro-Wilk Test
        # Opmerking: de Shapiro-Wilk test is beperkt tot 5000 waarnemingen
        if len(column_data) > 5000:
            column_data = column_data.sample(5000)
        stat, p = stats.shapiro(column_data)
        print(f'Shapiro-Wilk Test voor {column}: Statistiek={stat}, p-waarde={p}')

# Voorbeeld van hoe de functie te gebruiken
# analyze_data_distribution(df, headers)

Deze functie, 'describe_column', is ontworpen om essentiële statistieken en informatie te verstrekken over een specifieke kolom in een pandas DataFrame. Het geeft inzicht in de minimale en maximale waarden, evenals een overzicht van alle unieke waarden in de kolom en hun respectievelijke voorkomens. Deze functie is bijzonder nuttig voor initiële data-exploratie, waarbij een gedetailleerd begrip van individuele kolommen nodig is. Dit omvat het identificeren van het bereik van waarden, de verscheidenheid van data, en frequenties van specifieke waarden. Dergelijke informatie is cruciaal voor het bepalen van de juiste dataverwerkings- en analysestrategieën.


In [5]:
def describe_column(data, column_name, bin_size=10):
    # Vindt de minimale en maximale waarden van de gespecificeerde kolom
    min_value = data[column_name].min()
    max_value = data[column_name].max()
    
    # Toont de kolomnaam, minimum, en maximum waarden
    print(f"Column: {column_name}")
    print(f"Minimum value: {min_value}")
    print(f"Maximum value: {max_value}")
    
    # Creëer de bins
    bins = np.arange(min_value, max_value + bin_size, bin_size)
    
    # Classificeert de waarden in de gespecificeerde bins
    data['Bin'] = pd.cut(data[column_name], bins=bins, include_lowest=True)

    # Groepeert de data op basis van de bins en telt de waarden
    binned_data = data.groupby('Bin')[column_name].size()

    # Toont het aantal waarden in elke bin (exclusief bins met 0 waarden)
    print("Number of values per bin (excluding bins with 0 values):")
    for bin_range, count in binned_data.items():
        if count > 0:
            print(f"{bin_range}: {count} values")



This function 'route_per_week' plots truck routes for each day of the week within a specified week.
Parameters:

df: DataFrame containing the truck data.

start_date: String representing the start date of the week ('YYYY-MM-DD').

datetime_column: The name of the column in df containing datetime information.

In [15]:

def route_per_week(df, start_date, datetime_column='Vdatetime'):
   
    # Drop rows where any of these critical columns are missing
    filtered_df = df.dropna(subset=['Vdatetime', 'GPSLatitude', 'GPSLongitude'])

    # Drop rows where GPSLatitude or GPSLongitude are 0
    filtered_df = filtered_df[(filtered_df['GPSLatitude'] > 0) | (filtered_df['GPSLongitude'] > 0)]
    
    # Convert start_date to datetime
    start_date = pd.to_datetime(start_date)
    # Calculate end_date as 7 days after start_date
    end_date = start_date + pd.Timedelta(days=7)
    
    # Filter the DataFrame to the specified week
    filtered_df = filtered_df[(filtered_df[datetime_column] >= start_date) & (filtered_df[datetime_column] <= end_date)]

    # Drop rows where GPSLatitude or GPSLongitude are 0
    filtered_df = filtered_df[(filtered_df['GPSLatitude'] > 0) | (filtered_df['GPSLongitude'] > 0)]
    
    # Add a 'day_of_week' column to the filtered DataFrame
    filtered_df['day_of_week'] = filtered_df[datetime_column].dt.dayofweek
    
    # Set the style of the visualization
    sns.set(style="darkgrid")
    
    # Define a mapping from day numbers to day names
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
    # Plotting
    plt.figure(figsize=(10, 6))
    
    # Loop through each day of the week
    for day in range(7):
        # Filter the DataFrame for the day
        day_df = filtered_df[filtered_df['day_of_week'] == day]
        
        # Plot, using the day name in the label
        plt.plot(day_df['GPSLongitude'], day_df['GPSLatitude'], marker='o', linestyle='-', label=day_names[day])
    
    plt.title(f'RCV Routes in the Week Starting from {start_date.strftime("%Y-%m-%d")}')
    plt.xlabel('Longitude')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend(title='Day of Week')
    plt.show()


Onderstaande code doet in principe het zelfde als route_per_week alleen geeft het de output in een folium map weer.

In [16]:
def route_per_week_folium(df, start_date, datetime_column='Vdatetime'):

    # Drop rows where any of these critical columns are missing
    filtered_df = df.dropna(subset=['Vdatetime', 'GPSLatitude', 'GPSLongitude'])

    # Drop rows where GPSLatitude or GPSLongitude are 0
    filtered_df = filtered_df[(filtered_df['GPSLatitude'] > 0) | (filtered_df['GPSLongitude'] > 0)]

    # Convert start_date to datetime and calculate end_date
    start_date = pd.to_datetime(start_date)
    end_date = start_date + pd.Timedelta(days=7)
    
    # Filter DataFrame for the specified week
    filtered_df = filtered_df[(filtered_df[datetime_column] >= start_date) & (filtered_df[datetime_column] <= end_date)]
    
    # With 'GPSLatitude' and 'GPSLongitude'as coordinate columns
    filtered_df['day_of_week'] = filtered_df[datetime_column].dt.dayofweek
    
    # Initialize a map centered around the first point in your filtered DataFrame
    if not filtered_df.empty:
        start_coords = (filtered_df.iloc[0]['GPSLatitude'], filtered_df.iloc[0]['GPSLongitude'])
    else:
        # Default to a general location if DataFrame is empty
        start_coords = (0, 0)
    folium_map = folium.Map(location=start_coords, zoom_start=12)
    
    # Define colors for each day of the week
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightblue']
    
    # Plot each day's route
    for day in range(7):
        day_df = filtered_df[filtered_df['day_of_week'] == day]
        locations = day_df[['GPSLatitude', 'GPSLongitude']].values.tolist()
        if locations:
            folium.PolyLine(locations, color=colors[day], weight=2.5, opacity=1, tooltip=f'Day {day}').add_to(folium_map)
    
    return folium_map
